In [79]:
import streamlit as st
import folium
from streamlit_folium import st_folium
from geopy.geocoders import Nominatim
import osmnx as ox
import numpy as np 
import matplotlib.pyplot as plt
from folium import GeoJson, GeoJsonTooltip
import geopandas as gpd

In [2]:

address = "Skaldevägen 60"
POI_radius=500

geolocator = Nominatim(user_agent="Navigator")
location = geolocator.geocode(address)
lat, lon = location.latitude, location.longitude

In [6]:
m = folium.Map(location=[lat, lon], zoom_start=14)


In [113]:
folium.Marker([lat, lon], popup=address, icon=folium.Icon(color='red', icon='home')).add_to(m)
folium.Circle(
         location=[lat, lon],
         radius=POI_radius,  # in meters
         color='blue',       # outline color
         fill=True,
         fill_color='blue',  # fill color
         fill_opacity=0.2,   # transparency (0 = invisible, 1 = opaque)
         weight=2            # outline thickness
         ).add_to(m)



In [114]:
m

In [7]:
# Built environment: get POIs within 500m
tags = {
    'amenity':{ 'types':['cafe', 'restaurant', 'bar', 'fast_food', 'pub'],
               'color':'orange',
               'icon':'cutlery'
               },
    'shop': {'types':['supermarket', 'convenience', 'bakery'],
             'color':'green',
             'icon':'shopping-cart'
             }
}

In [9]:
#subset the dictionary so that it contains only the keys and types = readable in ox.features_from_point()
osm_tags = {key: info['types'] for key, info in tags.items()}
pois = ox.features_from_point((lat, lon), tags=osm_tags, dist=POI_radius)

In [56]:
osm_tags

{'amenity': ['cafe', 'restaurant', 'bar', 'fast_food', 'pub'],
 'shop': ['supermarket', 'convenience', 'bakery']}

In [27]:
pois[list(osm_tags.keys()) + ["geometry"pois[[*osm_tags.keys(), "geometry"]]]]

amenity         shop  \
element id                                     
node    248752304    restaurant          NaN   
        566435738           NaN  supermarket   
        566435740          cafe          NaN   
        1512612813   restaurant          NaN   
        1516347784   restaurant          NaN   
        1563539320   restaurant          NaN   
        11975133977  restaurant          NaN   
        12163401910         NaN       bakery   
way     137978548     fast_food          NaN   

                                                              geometry  
element id                                                              
node    248752304                            POINT (17.92894 59.32944)  
        566435738                            POINT (17.92864 59.32886)  
        566435740                            POINT (17.92794 59.32891)  
        1512612813                           POINT (17.92822 59.32942)  
        1516347784                           POINT (17.93867 59.32884)  
        1563539320                            POINT (17.93164 59.3313)  
        11975133977                          POINT (17.92904 59.32958)  
        12163401910                          POINT (17.93596 59.32807)  
way     137978548    POLYGON ((17.92767 59.32919, 17.92789 59.32915...

In [32]:
pois[[*osm_tags.keys(), "geometry"]]

amenity         shop  \
element id                                     
node    248752304    restaurant          NaN   
        566435738           NaN  supermarket   
        566435740          cafe          NaN   
        1512612813   restaurant          NaN   
        1516347784   restaurant          NaN   
        1563539320   restaurant          NaN   
        11975133977  restaurant          NaN   
        12163401910         NaN       bakery   
way     137978548     fast_food          NaN   

                                                              geometry  
element id                                                              
node    248752304                            POINT (17.92894 59.32944)  
        566435738                            POINT (17.92864 59.32886)  
        566435740                            POINT (17.92794 59.32891)  
        1512612813                           POINT (17.92822 59.32942)  
        1516347784                           POINT (17.93867 59.32884)  
        1563539320                            POINT (17.93164 59.3313)  
        11975133977                          POINT (17.92904 59.32958)  
        12163401910                          POINT (17.93596 59.32807)  
way     137978548    POLYGON ((17.92767 59.32919, 17.92789 59.32915...

In [42]:
keys=list(osm_tags.keys())
keys[0]
i=0

In [44]:
pois[keys[i]]

element  id         
node     248752304      restaurant
         566435738             NaN
         566435740            cafe
         1512612813     restaurant
         1516347784     restaurant
         1563539320     restaurant
         11975133977    restaurant
         12163401910           NaN
way      137978548       fast_food
Name: amenity, dtype: object

In [49]:
filtered_pois = pois[pois[keys[i]].notna() & pois['name'].notna()][[keys[i], 'name', 'geometry']]

In [50]:
filtered_pois

amenity                 name  \
element id                                             
node    248752304    restaurant     Nockeby Pizzeria   
        566435740          cafe       Nockeby bageri   
        1512612813   restaurant              Mezbaan   
        1516347784   restaurant             Appetito   
        1563539320   restaurant  Pizzeria Red Corner   
        11975133977  restaurant         PanPan Sushi   
way     137978548     fast_food       Nockebygrillen   

                                                              geometry  
element id                                                              
node    248752304                            POINT (17.92894 59.32944)  
        566435740                            POINT (17.92794 59.32891)  
        1512612813                           POINT (17.92822 59.32942)  
        1516347784                           POINT (17.93867 59.32884)  
        1563539320                            POINT (17.93164 59.3313)  
        11975133977                          POINT (17.92904 59.32958)  
way     137978548    POLYGON ((17.92767 59.32919, 17.92789 59.32915...

In [52]:
print(filtered_pois.iterrows())

<generator object DataFrame.iterrows at 0x0000015D0017EF00>


In [54]:
for i in range(len(keys)):
    feature_layer = folium.FeatureGroup(name=keys[i])
    filtered_pois = pois[pois[keys[i]].notna() & pois['name'].notna()]
    #for each POI of the current key (row in the filtered_poi
    for idx, row in filtered_pois.iterrows():
        #get longitude and latitude
        lon_, lat_ = row.geometry.centroid.xy
        #add a marker to the map
        folium.Marker(
            location=[lat_[0], lon_[0]],
            popup= f"<div style='font-size:12px; font-family:Arial; white-space:nowrap;'><b>{row.get(key,'N/A')}</b><br>{row.get('name')}",
            icon = folium.Icon(color=item['color'], icon=item['icon'])
            
            ).add_to(feature_layer)

    

In [55]:
m

In [89]:
tags_area = {'landuse': ['commercial', 'industrial', 'residential', 'retail', 'institutional', 'recreation_ground','greenery', 'forest']}

In [90]:
tags_area

{'landuse': ['commercial',
  'industrial',
  'residential',
  'retail',
  'institutional',
  'recreation_ground',
  'greenery',
  'forest']}

In [91]:
areas =ox.features_from_point((lat, lon), tags=tags_area, dist=POI_radius)

In [104]:
areas

geometry  \
element  id                                                              
relation 4871435     POLYGON ((17.93971 59.32743, 17.94008 59.32751...   
         5744315     POLYGON ((17.94243 59.32998, 17.94238 59.33011...   
         7015120     POLYGON ((17.93329 59.32404, 17.93349 59.3241,...   
way      22879910    POLYGON ((17.93501 59.33171, 17.93438 59.33245...   
         23315704    POLYGON ((17.94106 59.33307, 17.94104 59.33313...   
...                                                                ...   
         679336518   POLYGON ((17.92776 59.32936, 17.92818 59.33005...   
         680467022   POLYGON ((17.93864 59.32753, 17.93867 59.32749...   
         924657200   POLYGON ((17.93451 59.33008, 17.93457 59.33007...   
         1408634295  POLYGON ((17.92771 59.33296, 17.92625 59.33284...   
         1408634305  POLYGON ((17.93264 59.33299, 17.93356 59.33291...   

                         landuse           name       place leisure  \
element  id                                                           
relation 4871435          forest   Sångarlunden         NaN     NaN   
         5744315     residential      Bokhyllan  city_block     NaN   
         7015120          forest      Tunadalen         NaN     NaN   
way      22879910         forest  Nockebyskogen         NaN     NaN   
         23315704         forest            NaN         NaN     NaN   
...                          ...            ...         ...     ...   
         679336518   residential         Brevet  city_block     NaN   
         680467022        forest            NaN         NaN     NaN   
         924657200        forest            NaN         NaN     NaN   
         1408634295   industrial            NaN         NaN     NaN   
         1408634305   industrial            NaN         NaN     NaN   

                             type  
element  id                        
relation 4871435     multipolygon  
         5744315     multipolygon  
         7015120     multipolygon  
way      22879910             NaN  
         23315704             NaN  
...                           ...  
         679336518            NaN  
         680467022            NaN  
         924657200            NaN  
         1408634295           NaN  
         1408634305           NaN  

[122 rows x 6 columns]

In [95]:
colors = {
    'commercial': 'orange',
    'industrial': 'gray',
    'residential': 'lightblue',
    'retail': 'red',
    'institutional': 'purple',
    'recreation_ground': 'lightgreen',
    'greenery': 'green',
    'forest': 'darkgreen'
}

In [111]:
a1=areas[1:10]
a1

geometry  \
element  id                                                             
relation 5744315    POLYGON ((17.94243 59.32998, 17.94238 59.33011...   
         7015120    POLYGON ((17.93329 59.32404, 17.93349 59.3241,...   
way      22879910   POLYGON ((17.93501 59.33171, 17.93438 59.33245...   
         23315704   POLYGON ((17.94106 59.33307, 17.94104 59.33313...   
         55462860   POLYGON ((17.93807 59.33478, 17.938 59.33489, ...   
         61961066   POLYGON ((17.94372 59.32879, 17.94365 59.3286,...   
         61975811   POLYGON ((17.94463 59.33027, 17.94559 59.32998...   
         333293713  POLYGON ((17.94291 59.3328, 17.94372 59.33297,...   
         333293714  POLYGON ((17.94263 59.33275, 17.94229 59.33266...   

                        landuse           name       place leisure  \
element  id                                                          
relation 5744315    residential      Bokhyllan  city_block     NaN   
         7015120         forest      Tunadalen         NaN     NaN   
way      22879910        forest  Nockebyskogen         NaN     NaN   
         23315704        forest            NaN         NaN     NaN   
         55462860        forest            NaN         NaN     NaN   
         61961066        forest            NaN         NaN     NaN   
         61975811        forest            NaN         NaN     NaN   
         333293713  residential            NaN         NaN     NaN   
         333293714  residential            NaN         NaN     NaN   

                            type  
element  id                       
relation 5744315    multipolygon  
         7015120    multipolygon  
way      22879910            NaN  
         23315704            NaN  
         55462860            NaN  
         61961066            NaN  
         61975811            NaN  
         333293713           NaN  
         333293714           NaN

In [112]:
m = folium.Map(location=[lat, lon], zoom_start=14)
for landuse_type, group in areas.groupby('landuse'):
    color = colors.get(landuse_type, 'black')  # fallback color
    geojson = folium.GeoJson(
        group,
        name=landuse_type,
        tooltip=GeoJsonTooltip(fields=['landuse'])
    )
    geojson.add_to(m)
m

In [82]:
areas_proj = areas.to_crs(areas.estimate_utm_crs())

In [85]:
areas_proj

geometry  \
element  id                                                              
relation 5744315     POLYGON ((667394.095 6580494.989, 667390.427 6...   
way      333293713   POLYGON ((667407.312 6580810.202, 667452.485 6...   
         333293714   POLYGON ((667391.703 6580802.967, 667372.832 6...   
         333293715   POLYGON ((667321.698 6580777.003, 667286.044 6...   
         343355439   POLYGON ((667080.633 6580511.517, 667051.739 6...   
...                                                                ...   
         678963971   POLYGON ((666967.553 6580097.37, 667031.136 65...   
         678964013   POLYGON ((666804.907 6580421.308, 666811.176 6...   
         679336518   POLYGON ((666562.73 6580388.592, 666583.347 65...   
         1408634295  POLYGON ((666541.953 6580789.932, 666459.906 6...   
         1408634305  POLYGON ((666822.522 6580804.749, 666875.27 65...   

                         landuse        name       place          type  \
element  id                                                              
relation 5744315     residential   Bokhyllan  city_block  multipolygon   
way      333293713   residential         NaN         NaN           NaN   
         333293714   residential         NaN         NaN           NaN   
         333293715   residential         NaN         NaN           NaN   
         343355439   residential      Tavlan  city_block           NaN   
...                          ...         ...         ...           ...   
         678963971   residential  Allegorien  city_block           NaN   
         678964013   residential      Hymnen  city_block           NaN   
         679336518   residential      Brevet  city_block           NaN   
         1408634295   industrial         NaN         NaN           NaN   
         1408634305   industrial         NaN         NaN           NaN   

                          area_m2  
element  id                        
relation 5744315     19722.046769  
way      333293713    1513.423613  
         333293714    1474.390685  
         333293715    3538.799655  
         343355439    4498.698169  
...                           ...  
         678963971   13671.862371  
         678964013    5982.039470  
         679336518    2061.494472  
         1408634295   1657.082429  
         1408634305   1166.045049  

[97 rows x 6 columns]

In [84]:
areas_proj['area_m2'] = areas_proj.geometry.area

In [86]:
area_summary = areas_proj.groupby('landuse')['area_m2'].sum().reset_index()

In [87]:
area_summary

,landuse,area_m2
0,industrial,2823.127478
1,residential,733066.546392


In [143]:
tags0 = {
    'landuse': True,   # True → all landuse values
    'natural': True,   # all natural features
    'leisure': True,    # all leisure features
    'amenity':True,
    'shop':True,
    'office':True
}

In [144]:
all_features = ox.features_from_point((lat, lon),tags=tags0, dist=POI_radius)

In [162]:
all_features_proj = all_features.to_crs(all_features.estimate_utm_crs())
all_features_proj['area_m2'] = round(all_features_proj.geometry.area, 0)
area_summary = all_features_proj.groupby('landuse')['area_m2'].sum().reset_index()
area_summary["prop"] = area_summary["area_m2"]/sum(area_summary["area_m2"])
area_summary

,landuse,area_m2,prop
0,allotments,9784.0,0.008201
1,forest,429197.0,0.359739
2,grass,18203.0,0.015257
3,industrial,2823.0,0.002366
4,residential,733072.0,0.614437


In [149]:
industrial= all_features[all_features['landuse'] == 'industrial']

In [150]:
# Create base map
m = folium.Map(location=[lat, lon], zoom_start=14)

# Plot all polygons with same style
folium.GeoJson(
    data=industrial.__geo_interface__,
    style_function=lambda x: {
        'fillColor': 'orange',  # all polygons same color
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.5
    }
).add_to(m)
m